# Hadronic Baseline Calculator

## Detailed Analysis

This script performs a preliminary calculation required before the main dataset generation begins. It iterates through every pure hadronic Equation of State (EoS) model available in the library and calculates its maximum stable mass.

The resulting "baseline" masses are essential for the **Inverse Sampling** technique used later in the generation pipeline. When generating synthetic stars, the worker scripts scale these parent models using a homologous scaling factor ($\alpha$) to reach a specific target mass. These baselines provide the reference point ($M_{base}$) for that scaling.

## Physics and Math

The script identifies the maximum mass configuration for a given EoS by solving the Tolman-Oppenheimer-Volkoff (TOV) equations over a sequence of central pressures.

For a specific EoS model $i$, the maximum stable mass is defined as:

$$
M_{max, i} = \max_{P_c} M(P_c; \text{EoS}_i)
$$

where $M(P_c)$ is the stellar mass resulting from integrating the TOV equations with central pressure $P_c$.

In this specific script, the EoS models are evaluated in their "pure" form, meaning:
1.  **No Mixing:** The mixing weight is set to $w=1.0$.
2.  **No Scaling:** The homologous scaling factor is set to $\alpha=1.0$.

## Code Walkthrough

### 1. Library Retrieval
The script first retrieves the library of analytic core models and crust functions. It then initializes an empty dictionary to store the results.

```python
core_lib, crust_funcs = get_eos_library()
baselines = {}
```

### 2. Model Configuration and Transition Pressure
The script iterates through each model name in the library. It determines the appropriate crust-core transition pressure ($P_{trans}$). While most models use a standard default value derived from the SLy EoS, the "PS" model requires a specific higher transition pressure.

```python
# Determine the specific Crust-Core transition pressure.
p_trans = CONSTANTS['P_TRANS_PS'] if name == "PS" else CONSTANTS['P_TRANS_DEFAULT']

# Prepare the input tuple for the TOV solver.
# w=1.0 (no mixing), alpha=1.0 (no scaling)
eos_input = (f[0], f[1], f[0], f[1], 1.0, crust_funcs, 1.0, p_trans)
```

### 3. Solving for Maximum Mass
The `solve_sequence` function is called to integrate the TOV equations across a range of central pressures. This function returns the full sequence of stellar configurations and explicitly identifies the maximum mass ($max\_m$).

```python
# Solve the TOV sequence to find the maximum stable mass
_, max_m = solve_sequence(eos_input, is_quark=False)

if max_m > 1.0: 
    baselines[name] = max_m
```

The valid maximum masses are stored in the dictionary, which is returned for use by the parallel worker processes.